<a href="https://colab.research.google.com/github/RentoYabuki06/note-scraping/blob/main/%E3%80%90%E9%85%8D%E5%B8%83%E7%94%A8%E3%80%91NoteScraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ステップ0. 同じ階層に「.env」ファイルを作成する
作成するファイルの中には以下の情報を記載してください。

おそらくChatGPTなどのAIに聞くとスムーズに取得方法がわかります。
- SPREADSHEET_ID="あなたのGoogle Spread Sheet ID"
- SHEET_NAME="シート名（デフォルトは「シート1」）"
- COLUMN_TO_READ=2（何列目から読み取り始めるか）
- START_ROW=2（何行から読み取り始めるか）
- NOTION_API_KEY="あなたのNotion Internal Integration Secret Key"
- DATABASE_ID="接続したいNotion DBのID"

# ステップ1. 自身のGoogle Driveをマウント

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# ステップ2. 必要なパッケージのインストール
- ✅ gspread：Sheets操作用
- ✅ google-auth：認証用
- ✅ notion-client：Notion操作用
- ✅ BeautifulSoup4, requests：スクレイピング用
- ✅ python-dotenv：ローカルの環境変数読み取り用


In [ ]:
!pip install gspread google-auth notion-client beautifulsoup4 requests python-dotenv

# ステップ3. ローカル環境変数の読み込み
作業ディレクトリのみ修正して以下のプログラムを実行してください。

In [ ]:
import os
from dotenv import load_dotenv

# 作業ディレクトリを移動 (このColaboファイルと.envファイルがある場所を指定してください)
os.chdir('/content/drive/MyDrive/NoteScraping')

# .env ファイルをロード
load_dotenv()

# 読み込んだ環境変数を個別の変数に格納 (デバッグ用に出力)
SPREADSHEET_ID = os.getenv("SPREADSHEET_ID")
SHEET_NAME = os.getenv("SHEET_NAME")
COLUMN_TO_READ = os.getenv("COLUMN_TO_READ", "2")
START_ROW = os.getenv("START_ROW", "2")
NOTION_API_KEY = os.getenv("NOTION_API_KEY")
DATABASE_ID = os.getenv("DATABASE_ID")
GSHEET_SCOPES_STR = os.getenv("GSHEET_SCOPES", "https://www.googleapis.com/auth/spreadsheets,https://www.googleapis.com/auth/drive.readonly")
GSHEET_SCOPES = [scope.strip() for scope in GSHEET_SCOPES_STR.split(',')]

print("DEBUG: Loaded environment variables:")
print(f"  SPREADSHEET_ID: {SPREADSHEET_ID}")
print(f"  SHEET_NAME: {SHEET_NAME}")
print(f"  COLUMN_TO_READ: {COLUMN_TO_READ}")
print(f"  START_ROW: {START_ROW}")
print(f"  NOTION_API_KEY: {NOTION_API_KEY}")
print(f"  DATABASE_ID: {DATABASE_ID}")
print(f"  GSHEET_SCOPES: {GSHEET_SCOPES}")

# ステップ4. スクレイピング & notion DBへの書き込み実行
以下のプログラムをそのまま実行してください。

Notion DB側のカラム名に誤りがある場合は適切に動作しません。（大文字小文字スペースなどにご注意ください）

In [ ]:
from google.colab import auth
import gspread
from google.auth import default
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from dotenv import load_dotenv
import os
import requests
from bs4 import BeautifulSoup
from notion_client import Client
import re

# === Google Sheets APIクライアントの初期化 ===
def initialize_gsheets_client(scopes):
    auth.authenticate_user()
    creds, _ = default(scopes=scopes)
    return gspread.authorize(creds)

# === スプレッドシートからURLを読み取る ===
def read_urls_from_sheet(gc, spreadsheet_id, sheet_name, column_to_read, start_row):
    try:
        spreadsheet = gc.open_by_key(spreadsheet_id)
        worksheet = spreadsheet.worksheet(sheet_name)
        return worksheet.col_values(column_to_read)[start_row - 1:]
    except gspread.SpreadsheetNotFound:
        print(f"スプレッドシートが見つかりませんでした: {spreadsheet_id}")
        return None
    except gspread.WorksheetNotFound:
        print(f"シート '{sheet_name}' が見つかりませんでした。")
        return None
    except Exception as e:
        print(f"スプレッドシートからのURL読み取り中にエラーが発生しました: {e}")
        return None

# === 記事データをスクレイピング ===
def scrape_note_data(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    date_tag = soup.find('time')
    publish_date = date_tag['datetime'] if date_tag else None

    title_tag = soup.find('title')
    title = title_tag.text.strip() if title_tag else '無題'

    like_button = soup.select_one('button.o-noteLikeV3__count')
    like_count = int(like_button.text.strip()) if like_button and like_button.text.strip().isdigit() else 0

    article_text = soup.get_text()
    hashtags = list(set(re.findall(r'#(\S+)', article_text)))

    return {
        'url': url,
        'publish_date': publish_date,
        'likes': like_count,
        'title': title,
        'hashtags': hashtags
    }

# === Notionに登録する ===
def create_notion_page(notion, database_id, data):
    properties = {
        "Title": {
            "title": [{"text": {"content": data['title']}}]
        },
        "Like": {
            "number": data['likes']
        },
        "View": {
            "number": 0  # View数は今回はスキップ
        },
        "URL": {
            "url": data['url']
        },
        "Charnum": {
            "number": 0  # 文字数カウント未実装
        },
        "Category": {
            "multi_select": [{"name": tag} for tag in data['hashtags']] or [{"name": "note"}]
        }
    }

    if data.get('publish_date'):
        properties["Publish"] = {
            "date": {"start": data['publish_date']}
        }

    try:
        notion.pages.create(
            parent={"database_id": database_id},
            properties=properties
        )
        return True
    except Exception as e:
        print(f"🔥 Notionへの登録エラー (URL: {data['url']}): {e}")
        return False

# === メイン実行 ===
if __name__ == '__main__':
    notion = Client(auth=NOTION_API_KEY)
    gc = initialize_gsheets_client(GSHEET_SCOPES)

    if gc:
        urls_to_process = read_urls_from_sheet(gc, SPREADSHEET_ID, SHEET_NAME, int(COLUMN_TO_READ), int(START_ROW))

        if urls_to_process:
            for article_url in urls_to_process:
                print(f"🚀 スクレイピング開始: {article_url}")
                try:
                    article_data = scrape_note_data(article_url)
                    if create_notion_page(notion, DATABASE_ID, article_data):
                        print(f"✅ 登録完了: {article_data['title']}")
                except requests.exceptions.RequestException as e:
                    print(f"⚠️ スクレイピングエラー (URL: {article_url}): {e}")
                except Exception as e:
                    print(f"🔥 予期せぬエラー (URL: {article_url}): {e}")
        else:
            print("処理するURLが見つかりませんでした。")

    print("✨ 全てのURLの処理が完了しました。")

# ステップ5. notion DBに反映されているか確認！
Colaboでの処理はここまでです。

Notion DBに適切に反映されているか確認してみましょう！

もしエラーが出る場合は AIに聞いてみるなどして解決を図りましょう！